In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score
from IPython.display import clear_output


In [2]:
#preparing the data : Given a file, the function prepares 2 arrays
"""
feature_set : each element is an array of 50 readings
targets : each element is a label corrisponding to the readings with the same index in feature_set
"""
def getSamplesFromFiles(filesNames=["mHealth_ECGProcessed_subject3.log"],readingsPerSample=50,trainPercentage=0.8):
    targets  = np.array([])
    feature_set= np.array([])
    for fileName in filesNames:
        df = pd.read_csv(fileName, header=None, delim_whitespace=True)
        for i in range(int(df.shape[0]/readingsPerSample)):
            if df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean() in range(1,13):
                feature_set=np.append(feature_set,[df[1][i*readingsPerSample:i*readingsPerSample+readingsPerSample]])
                targets=np.append(targets,df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean()-1)

    feature_set=feature_set.reshape(int(feature_set.shape[0]/readingsPerSample),readingsPerSample)
    total = len(feature_set)
    return feature_set[0:int(trainPercentage*total)], targets[0:int(trainPercentage*total)], feature_set[int(trainPercentage*total):], targets[int(trainPercentage*total):]

In [6]:
#preparing the data : Given a file, the function prepares 2 arrays
"""
feature_set : each element is an array of 50 readings
targets : each element is a label corrisponding to the readings with the same index in feature_set
"""
def getSamplesFromFiles(filesNames=["mHealth_ECGProcessed_subject3.log"],readingsPerSample=50):
    targets  = np.array([])
    feature_set= np.array([])
    for fileName in filesNames:
        df = pd.read_csv(fileName, header=None, delim_whitespace=True)
        for i in range(int(df.shape[0]/readingsPerSample)):
            if df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean() in range(1,13):
                feature_set=np.append(feature_set,[df[1][i*readingsPerSample:i*readingsPerSample+readingsPerSample]])
                targets=np.append(targets,df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean()-1)

    feature_set=feature_set.reshape(int(feature_set.shape[0]/readingsPerSample),readingsPerSample)
#     one_hot_labels=one_hot_labels.reshape(int(one_hot_labels.shape[0]/12),12)
    return feature_set, targets
def unison_shuffled_copies(a, b,seed=1):
    assert len(a) == len(b)
    np.random.seed(seed)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [5]:
def dummy(filesNames=["mHealth_ECGProcessed_subject3.log"],readingsPerSample=50):
    targets  = np.array([])
    feature_set= np.array([])
    for fileName in filesNames:
        df = pd.read_csv(fileName, header=None, delim_whitespace=True)
        for i in range(int(df.shape[0]/readingsPerSample)):
            if df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean() in range(1,13):
                feature_set=np.append(feature_set,[df[1][i*readingsPerSample:i*readingsPerSample+readingsPerSample]])
                targets=np.append(targets,df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean()-1)

    feature_set=feature_set.reshape(int(feature_set.shape[0]/readingsPerSample),readingsPerSample)
#     one_hot_labels=one_hot_labels.reshape(int(one_hot_labels.shape[0]/12),12)
    return feature_set[:10], targets[:10]

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
# Load the data:
# X, y = getSamplesFromFiles(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log","mHealth_ECGProcessed_subject10.log"],100)
# feature_set,targets,test_feature_set,test_targets=getSamplesFromFiles(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log","mHealth_ECGProcessed_subject10.log"],100,0.7)
feature_set,targets=getSamplesFromFiles(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log","mHealth_ECGProcessed_subject10.log"],100)
# feature_set,targets=dummy(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log","mHealth_ECGProcessed_subject10.log"],100)
feature_set,targets=unison_shuffled_copies(feature_set,targets)
X = torch.from_numpy(feature_set).float()
y = torch.from_numpy(targets).long()
# feature_set=torch.from_numpy(feature_set).float()
# targets=torch.from_numpy(targets).long()

# Split in 5 folds:
n = len(X)
kf = KFold(n_splits=10)
fold = 0
model = Net()
for train_idx, test_idx in kf.split(X):
    fold += 1
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
#     clf = LogisticRegression().fit(X_train, y_train)
#     score = clf.score(X_test, y_test)
    m = trainModel(X_train,y_train,10000)
    m.eval() # prep model for evaluation
    # forward pass: compute predicted outputs by passing inputs to the model 
    y_pred = m(X_test)
    score=precision_score(y_test, y_pred.argmax(1), average='macro')
    print("FOLD : ",fold)
    print("NN score: %.3f" % (score))

    

C:\Users\pcroot\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


FOLD :  1
NN score: 0.015
FOLD :  2
NN score: 0.043
FOLD :  3
NN score: 0.004
FOLD :  4
NN score: 0.011
FOLD :  5
NN score: 0.018
FOLD :  6
NN score: 0.008
FOLD :  7
NN score: 0.010
FOLD :  8
NN score: 0.009
FOLD :  9
NN score: 0.025
FOLD :  10
NN score: 0.006


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 60
        hidden_2 = 30
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(100, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 12)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
#         x = x.view(-1, 50)
#         print('XXXXXXXXXXXXX',x.shape,'\n',x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = self.fc3(x)        
        return x

# initialize the NN
model = Net()

In [222]:
# feature_set,targets,test_feature_set,test_targets=getSamplesFromFiles(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log","mHealth_ECGProcessed_subject10.log"],100,0.7)
feature_set,targets=getSamplesFromFiles(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log","mHealth_ECGProcessed_subject10.log"],100)
# feature_set,targets=dummy(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log","mHealth_ECGProcessed_subject10.log"],100)
feature_set,targets=unison_shuffled_copies(feature_set,targets)

In [198]:
feature_set.shape
test_feature_set.shape
feature_set.shape

(1340, 100)

In [8]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [9]:
def trainModel(feature_set, targets,n_epochs=50000,prints=False):
    
    # number of epochs to train the model
#     n_epochs = 10000

    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf # set initial "min" to infinity

    for epoch in range(n_epochs):
        # monitor training loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train() # prep model for training

        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(feature_set)
        # calculate the loss
        loss = criterion(output, targets)
    #     print('XXXXXXXXXXXXXX',output[0])
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()

        if prints :
            print('Epoch: {} \tTraining Loss: {:.6f} '.format(
                epoch+1, 
                loss.item()
                ))
        
    return model


2

tensor([15.5387, -2.0145, -0.8594, -1.1398, -0.6586, -1.2397, -3.2278, -0.6465,
         0.5383, -0.2075, -2.1294, -0.1833], grad_fn=<SelectBackward>)

tensor([[ 1.0757e+00, -1.3174e-02, -2.2016e-01, -2.4896e-03, -1.1752e-01,
          4.2343e-02, -1.3989e-01, -2.3361e-01, -1.6352e-03,  3.9831e-05,
         -6.8518e-02, -7.8620e-02]], grad_fn=<AddmmBackward>)
tensor(1.0757, grad_fn=<MaxBackward1>)


tensor(0)

In [9]:
def accuracy(out, target):
    t = 0
    for o in range(len(out)):
        if int((output[o] == output[o].max()).nonzero()[0][0]) == int(target[o]):
            t+=1
        print(target[o])
    print(100*t/len(out))
        

In [182]:
accuracy(output,targets)

tensor(7)
tensor(1)
tensor(4)
tensor(2)
tensor(2)
tensor(10)
tensor(3)
tensor(9)
tensor(1)
tensor(1)
tensor(1)
tensor(6)
tensor(3)
tensor(1)
tensor(7)
tensor(1)
tensor(0)
tensor(2)
tensor(2)
tensor(8)
tensor(4)
tensor(0)
tensor(2)
tensor(11)
tensor(3)
tensor(0)
tensor(2)
tensor(3)
tensor(4)
tensor(3)
tensor(9)
tensor(1)
tensor(10)
tensor(9)
tensor(10)
tensor(11)
tensor(0)
tensor(5)
tensor(0)
tensor(4)
tensor(11)
tensor(5)
tensor(6)
tensor(0)
tensor(5)
tensor(3)
tensor(5)
tensor(1)
tensor(7)
tensor(10)
tensor(7)
tensor(3)
tensor(4)
tensor(8)
tensor(0)
tensor(5)
tensor(3)
tensor(7)
tensor(10)
tensor(2)
tensor(9)
tensor(3)
tensor(5)
tensor(6)
tensor(2)
tensor(11)
tensor(4)
tensor(10)
tensor(4)
tensor(10)
tensor(7)
tensor(7)
tensor(2)
tensor(7)
tensor(4)
tensor(9)
tensor(6)
tensor(3)
tensor(8)
tensor(2)
tensor(1)
tensor(5)
tensor(3)
tensor(9)
tensor(1)
tensor(10)
tensor(9)
tensor(10)
tensor(8)
tensor(11)
tensor(4)
tensor(5)
tensor(2)
tensor(7)
tensor(10)
tensor(8)
tensor(6)
tensor(10)
tens

tensor(5)
tensor(1)
tensor(6)
tensor(2)
tensor(9)
tensor(1)
tensor(9)
tensor(0)
tensor(9)
tensor(7)
tensor(4)
tensor(5)
tensor(3)
tensor(4)
tensor(4)
tensor(5)
tensor(2)
tensor(1)
tensor(10)
tensor(7)
tensor(7)
tensor(7)
tensor(0)
tensor(10)
tensor(9)
tensor(7)
tensor(3)
tensor(7)
tensor(4)
tensor(9)
tensor(9)
tensor(6)
tensor(10)
tensor(5)
tensor(1)
tensor(5)
tensor(5)
tensor(9)
tensor(5)
tensor(0)
tensor(7)
tensor(1)
tensor(8)
tensor(0)
94.1044776119403


In [11]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(12))
class_total = list(0. for i in range(12))

test_feature_set=torch.from_numpy(test_feature_set).float()
test_targets=torch.from_numpy(test_targets).long()

model.eval() # prep model for evaluation


# forward pass: compute predicted outputs by passing inputs to the model 
output = model(test_feature_set)
# calculate the loss
loss = criterion(output, test_targets)
# update test loss 
test_loss = loss.item()
# convert output probabilities to predicted class
_, pred = torch.max(output, 1)
# compare predictions to true label
correct = np.squeeze(pred.eq(test_targets.data.view_as(pred)))
# calculate test accuracy for each object class
for i in range(len(test_targets)):
    label = test_targets.data[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1

# calculate and print avg test loss
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(12):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 5.568794

Test Accuracy of     0: 10% ( 3/30)
Test Accuracy of     1: 13% ( 4/30)
Test Accuracy of     2: 15% ( 5/32)
Test Accuracy of     3:  8% ( 5/60)
Test Accuracy of     4: 16% ( 5/30)
Test Accuracy of     5: 16% ( 9/54)
Test Accuracy of     6: 11% ( 6/52)
Test Accuracy of     7:  8% ( 5/56)
Test Accuracy of     8: 18% (11/59)
Test Accuracy of     9: 13% ( 8/59)
Test Accuracy of    10: 33% (20/60)
Test Accuracy of    11:  0% ( 0/18)

Test Accuracy (Overall): 15% (81/540)


In [12]:
accuracy(output,test_targets)

tensor(2)
tensor(2)
tensor(2)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(3)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
